**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
from PIL import Image
from torchvision import models
from torchvision import transforms
from skimage.transform import resize
import matplotlib.pyplot as plt
import torch
import torchvision

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/djnjkz456tbgfnk/lion.png?dl=1", directory="data")
download_file_maybe_extract("https://www.dropbox.com/s/ma25i7w3jpqex2a/imagenet_classes?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

with open("data/imagenet_classes", "r") as file:
    classes = [c[:-1] for c in file.readlines()]

def decode_proba(proba, top=5):
    if isinstance(proba, torch.Tensor):
        proba = proba.cpu().numpy()
        
    proba = proba.ravel()
    ind = np.argsort(proba)
    
    for c in reversed(ind[-top:]):
        print("{}:\t{} ({})".format(
            np.array2string(proba[c], precision=5,
                            suppress_small=False),
            classes[c], c))

class PermuteTransform:
    def __init__(self, *dims):
        self.dims = dims

    def __call__(self, x):
        return x.permute(*self.dims)

class UnsqueezeTransform:
    def __init__(self, dim=0):
        self.dim = dim

    def __call__(self, x):
        return x.unsqueeze(self.dim)

class SqueezeTransform:
    def __init__(self, dim=0):
        self.dim = dim

    def __call__(self, x):
        return x.squeeze(self.dim)

class FromTensorTransform:
    def __call__(self, x):
        return x.detach().cpu().numpy()

class ClampTransform:
    def __init__(self, min=0., max=1.):
        self.min = min
        self.max = max

    def __call__(self, x):
        if not self.min is None:
            x = torch.clamp_min(x, self.min)
        
        if not self.max is None:
            x = torch.clamp_max(x, self.max)
        
        return x

class ToDeviceTransform:
    def __init__(self, device):
        self.device = device

    def __call__(self, x):
        return x.to(self.device)

## Protivnícke príklady

Tento notebook ukazuje jednu relatívne jednoduchú metódu na generovanie protivníckych príkladov.

Začneme tým, že si načítame ResNet architektúru s 50 vrstvami predtrénovanú na ImageNet-e. Sieť očakáva na vstupe obrázky rozmeru 224x224 a dokáže ich klasifikovať do jednej z 1000 tried (ich zoznam je v súbore data/classes a tiež sa zobrazí v kóde nižšie).



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=weights).to(device)

Ako obvykle, musíme sa uistiť, že vstupné obrázky predspracujeme spôsobom analogickým tomu, akým boli predspracované pri trénovaní zvolených váh. V tomto prípade preberieme len normalizáciu, pretože protivnícke príklady generované základnou metódou, ktorú budeme aplikovať, sú pomerne krehké a orezanie či zmena veľkosti by ich mohli poškodiť.

Budeme tiež potrebovať opačnú operáciu – pre prípad keď už získame protivnícky príklad a budeme ho chcieť zobraziť. Na tento účel si definujemem transformácie `normalize` a `denormalize` na základe normalizácie z `weights.transforms()`.

Nakoniec pridáme niekoľko ďalších transformácií, ako je permutácia rozmerov, aby sme sa dostali z formátu (šírka, výška, kanály) do formátu (kanály, šírka, výška) a tiež operácie `squeeze` a `unsqueeze`, ktoré odstránia či pridajú dávkový rozmer a podobne. To isté by sa dalo realizovať aj mimo `image_transform` a `image_detransform`, ale týmto spôsobom môžeme udržať všetky transformácie pohromade a náš kód bude o niečo prehľadnejší.



In [ ]:
weights_transforms = weights.transforms()

normalize = transforms.Normalize(
    mean=weights_transforms.mean,
    std=weights_transforms.std
)

denormalize = transforms.Normalize(
    mean=[-tm/sm for tm, sm in zip(weights_transforms.mean, weights_transforms.std)],
    std=[1.0/ts for ts in weights_transforms.std]
)

image_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    normalize,
    UnsqueezeTransform(),
    ToDeviceTransform(device)
])

image_detransform = torchvision.transforms.Compose([
    SqueezeTransform(),
    denormalize,
    PermuteTransform(1, 2, 0),
    ClampTransform(0., 1.),
    FromTensorTransform()
])

### Parametre

Tu si vyberieme cieľovú triedu: t.j. triedu do ktorej sa budeme snažiť, aby bol náš obrázok nesprávne klasifikovaný.



In [ ]:
# target_class = 231 # collie
# target_class = 413 # assault rifle
# target_class = 847 # tank
target_class = 409 # analog clock

Ak chcete získať zoznam všetkých tried, odkomentujte a spustite nasledujúcu bunku.



In [ ]:
# for ic, c in enumerate(classes):
#     print("{}:\t{}".format(ic, c))

### Načítanie a predspracovanie originálneho obrázka

Ďalej si načítame a zobrazíme originálny obrázok.



In [ ]:
img = plt.imread("data/lion.png")
plt.imshow(img); plt.axis('off');

Pomocou `image_transform` aplikujeme predspracovanie, ktoré naša predtrénovaná sieť očakáva. Následne dáme predspracovaný obrázok na vstup siete a zobrazíme 5 predikcií s najvyššími pravdepodobnosťami.



In [ ]:
img_t = image_transform(img)

In [ ]:
model.eval()
with torch.no_grad():
    y_logit = model(img_t)
    y_proba = torch.nn.functional.softmax(y_logit, dim=1)

In [ ]:
decode_proba(y_proba)

### Konštrukcia chybovej funkcie

Ďalším krokom je skonštruovať chybovú funkciu, ktorej minimalizáciou získame protivnícky obrázok. Keďže výsledkom optimalizácie bude protivnícky obrázok, vytvoríme si preň teraz osobitný tenzor. Vzhľadom na to, že jedným z kritérií bude, aby sa obrázok čo najviac podobal na originál, bude samozrejme rozumné, aby sme ho inicializovali tak, že originál okopírujeme.



In [ ]:
adv_t = img_t.clone().detach().requires_grad_(True)

Keď sme vytvorili protivnícky tenzor, obalíme ako tenzor (typu `long`) aj cieľovú triedu a zabezpečíme, aby sa výsledok preniesol na korektné zariadenie.



In [ ]:
target_class_t = torch.as_tensor([target_class], dtype=torch.long).to(device)

Pri výpočte chyby:

* Dáme protivnícky príklad na vstup siete a vypočítame jej výstup `y`;
* Chceme, aby bol vstup nesprávne klasifikovaný do triedy `target_class_t`, preto zostavíme klamové kritérium ako krížovú entropiu s parametrami `y` a `target_class_t` (pripomeňme, že krížovú entropiu používame aj keď trénujeme sieť, aby predikovala určité triedy);
* Skonštruujeme podobnostné kritérium ako $L^1$ vzdialenosť medzi protivníckym obrázkom a originálom;
* Obe chybové kritériá sčítame.


In [ ]:
def compute_loss():
    y = model(adv_t)
    deception_loss = torch.nn.functional.cross_entropy(y, target_class_t)
    similarity_loss = torch.nn.functional.l1_loss(adv_t, img_t)
    loss = deception_loss + similarity_loss
    return loss

### Optimalizácia

Vytvoríme optimalizátor a nastavíme parametre, ktoré bude optimalizovať: v našom prípade tenzor `adv_t`.



In [ ]:
optimizer = torch.optim.Adam([adv_t])

Definujeme funkciu, ktorú optimalizátor v každom kroku spustí:

* Vynulovanie gradientov z predchádzajúceho kroku.
* Výpočet chybovej funkcie.
* Spätné šírenie gradientov.
Aktualizáciu parametrov bude samozrejme riešiť samotný optimalizátor.



In [ ]:
def opti_step():
    optimizer.zero_grad()
    loss = compute_loss()
    loss.backward()
    return loss

Optimalizátor necháme niekoľko epoch bežať a zobrazujeme chyby.



In [ ]:
for epoch in range(500):
    optimizer.step(opti_step)
    if epoch % 50 == 0:
        print("Epoch {}; loss {}.".format(epoch, compute_loss().item()))

### Zobrazenie protivníckeho príkladu

Výsledný protivnícky príklad spracujeme, aby sme ho transformovali z tenzoru naspäť na prirodzený obrázok, ktorý sa dá vizualizovať. Protivnícky príklad tiež vložíme na vstup siete, aby sme sa presvedčili, či bude naozaj nesprávne klasifikovaný. Ak všetko prebehlo správne, bude teraz obrázok klasifikovaný ako analógové hodiny alebo nejaká iná cieľová trieda, ktorú sme si zvolili.



In [ ]:
adv = image_detransform(adv_t)

with torch.no_grad():
    y_logit = model(image_transform(adv))
    y_proba = torch.nn.functional.softmax(y_logit, dim=1)

decode_proba(y_proba)

Vykreslíme vedľa seba oba: pôvodný aj protivnícky obrázok.



In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=[10, 6])

axes[0].imshow(img)
axes[0].axis('off')
axes[0].set_title("the original image")

axes[1].imshow(adv)
axes[1].axis('off')
axes[1].set_title("the adversarial example");

Obrázky budú od seba vizuálne neodlíšiteľné. Aby sme ukázali, že skutočne nie sú rovnaké a v čom sa líšia, vypočítame a zobrazíme absolútne pixelové rozdiely medzi nimi (pričom priemerujeme cez farebné kanály).



In [ ]:
diff = np.abs(img - adv).mean(axis=-1)
plt.imshow(diff, cmap='Greys')
plt.axis('off')
plt.colorbar(label="pixel-wise difference (range [0, 1])");

---
### Úloha 1: Iný obrázok a cieľová trieda

**Aplikujte ten istý postup na iný obrázok a cieľovú triedu.** 

Poznámka: Nové obrázky môžete uploadovať **priamo cez notebook-ové rozhranie**  alebo alternatívne pomocou:

```
from google.colab import files
content_img = files.upload()
filename = list(content_img)[0]
```
---
